In [1]:
#导入模块
from astropy.io import fits
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from scipy.optimize import curve_fit
from skimage import draw
import glob, os
from pprint import pprint
import math
from datetime import datetime

from sklearn.cluster import KMeans


In [2]:

#读取文件
def readchase(file):
    from astropy.io import fits
    hdu = fits.open(file)
    try:
        data = hdu[0].data.astype(np.float32)
        header = hdu[0].header
        
    except:
        header = hdu[1].header
        data = hdu[1].data
        
    if len(data.shape) != 3:
        raise TypeError('file ', str(file), 'is not Chase\'s file, please use other function to read.')
        
    hdu_time = datetime.strptime(header['DATE_OBS'], "%Y-%m-%dT%H:%M:%S")
    if hdu_time < datetime.strptime('2023-04-18', "%Y-%m-%d"):
        cy = header['CRPIX1']
        cx = header['CRPIX2']
    else:
        cx = header['CRPIX1']
        cy = header['CRPIX2']
    

    #改变数组大小、日心位置
    data = data[:, int(cy-1023):int(cy+1025), int(cx-1023):int(cx+1025)]
    if data.shape != (118,2048,2048):
        raise TypeError('Chase file ', file, 'is corrupted, please check.')
    
    cx = 1023 + cx - int(cx)
    cy = 1023 + cy - int(cy)
    header['CRPIX1'] = 1023 + cx - int(cx)
    header['CRPIX2'] = 1023 + cy - int(cy)
    header['NAXIS1'] = 2048
    header['NAXIS2'] = 2048
    
    return data, header


In [3]:
#临边昏暗效应拟合
#函数定义

#定义距离函数
def dist(x1,y1, x2,y2):
    return abs(((x1-x2)**2 + (y1-y2)**2)**0.5)

#获取拟合数据
def get_f_r(line, data):
    f_r = []
    for element in line:
        x,y = element
        r = ((x-cx)**2 + (y-cy)**2)**0.5
        if r <= 900:#限制拟合半径
            f = data[y,x]
            f_r.append((r,f))
    
    f_r = sorted(f_r, key = lambda x:x[0])
    return np.array(f_r)

#拟合函数：cos函数
def func_cos(x,a,b,c):
    try:
        cos = abs((1 - (x/rsun)**2)**0.5)
    except:
        cos = 0
    f = a + b * cos + c * cos**2
    return f

def func(X,a,b,c):
    x = X[0]
    y = X[1]
    f = func_cos(x,a,b,c)
    return y/f

#筛选异常点
def select_d_num(y_pre,alpha = 3):
    
    d_num = []
    std = np.std(y_pre)
    
    for i in range(len(y_pre)):
        if abs(y_pre[i] - 1) > (alpha*std):
            d_num.append(i)
            
    return d_num

#实现函数拟合
def func_fit(func, data, L = 100):
    
    d_num = []
    p0 = [1,1,1]
    
    for i in range(L):
        data = np.delete(data, d_num, axis = 1)
        y = np.ones(shape = data.shape[1])
        
        popt, pcov = curve_fit(f = func, xdata = data, ydata = y, p0 = p0)
        p0 = popt
        y_pre = func(data, *popt)
        
        d_num = select_d_num(y_pre)
        
        if len(d_num) == 0:
            break
        d_point = data[:,d_num]
        
    return popt


In [4]:
#获得系数矩阵（用矩阵来算比遍历快一千倍！尽量别用遍历）
def get_coefmat(popt, cx, cy):
    
    a,b,c = popt
    rlim = 880

    from numpy import fromfunction

    def func_coef(i,j):
        r = dist(i,j,cx,cy)
        try:
            cos = abs((1 - (r/rsun)**2)**0.5)
        except:
            cos = 0
        f = a + b * cos + c * cos**2
        return (a+b+c)/f

    coef_mat = fromfunction(func_coef, (2048,2048))
    coef_mat = np.nan_to_num(coef_mat)

    disk = np.zeros(shape = (2048,2048))
    disk[draw.disk((cx,cy), rlim, shape = (2048,2048))] = 1

    coef_mat = coef_mat * disk
    
    return coef_mat

def kmeans_process(hdu_data, hdu_header):
    
    print('data processing')
    global cx,cy,rot,rsun
    cx = hdu_header['CRPIX2']
    cy = hdu_header['CRPIX1']
    rot = hdu_header['INST_ROT'] * math.pi / 180
    rsun = hdu_header['R_SUN']

    cos = math.cos(rot)*950
    sin = math.sin(rot)*950
    line = np.array(draw.line(int(cx-cos),int(cy-sin), int(cx+cos),int(cy+sin))).T
    
    ndata = np.zeros(shape = (118,2048,2048))
    
    poptlist = []
    for w in range(118):
        fdata = hdu_data[w]
        f_r = get_f_r(line, fdata)
        popt = func_fit(func, f_r.T)
        coef_mat = get_coefmat(popt, cx, cy)
        ndata[w] = fdata * coef_mat
        
    
    ndata = ndata / func_cos(0, *poptlist[69])
    
    print('file done')
    
    return ndata


In [6]:
file_path = "./chasefile"

file_list = glob.glob(os.path.join(file_path, "*HA.fits"))
pprint(file_list)

['./chasefile/RSM20230628T005007_0000_HA.fits',
 './chasefile/RSM20230728T003814_0000_HA.fits',
 './chasefile/RSM20230304T011852_0000_HA.fits',
 './chasefile/RSM20230208T000808_0000_HA.fits',
 './chasefile/RSM20230101T005240_0000_HA.fits',
 './chasefile/RSM20221226T005329_0000_HA.fits',
 './chasefile/RSM20221222T002215_0000_HA.fits',
 './chasefile/RSM20221230T000020_0000_HA.fits',
 './chasefile/RSM20230320T000022_0000_HA.fits',
 './chasefile/RSM20230724T000834_0000_HA.fits',
 './chasefile/RSM20230224T002424_0000_HA.fits',
 './chasefile/RSM20230204T011324_0000_HA.fits',
 './chasefile/RSM20230129T011610_0000_HA.fits',
 './chasefile/RSM20230212T003736_0000_HA.fits',
 './chasefile/RSM20230228T005025_0000_HA.fits',
 './chasefile/RSM20230322T010128_0000_HA.fits',
 './chasefile/RSM20230503T000439_0000_HA.fits',
 './chasefile/RSM20221218T000016_0000_HA.fits',
 './chasefile/RSM20221206T001443_0000_HA.fits',
 './chasefile/RSM20230521T012751_0000_HA.fits',
 './chasefile/RSM20230612T002214_0000_HA

In [7]:
print(len(file_list))

120


In [14]:
import joblib
from dateutil.parser import parse
from datetime import datetime

k = 30

error_list = []

for i in range(len(file_list)):
    try:
        starttime = datetime.now()

        hdu_file = file_list[i]
        test_data, test_header = readchase(hdu_file)
        test_ndata = kmeans_process(test_data, test_header)

        test_ha = test_data[1]
        test_tdata = test_ndata.reshape(118,-1).T

        kmodel = KMeans(n_clusters = k)
        kmodel.fit(test_tdata)
        kresult = kmodel.labels_
        kmap = np.array(kresult, dtype = float).reshape(2048,2048)

        hdu_date = parse(hdu_file.split("RSM")[1][:8]).strftime("%Y_%m%d")

        plt.imsave(f'./kmeans_result_30/imgs/{hdu_date}.png', test_ha, vmin = 0,
                   vmax = test_ha.mean()*3, cmap = 'gray', origin = 'lower')
        plt.imsave(f'./kmeans_result_30/ha/{hdu_date}.png', test_ha, vmin = 0,
                   vmax = test_ha.mean()*3, cmap = 'afmhot', origin = 'lower')


        joblib.dump(kmodel, f'./model_30/kmodel_{hdu_date}.pkl')

        for j in range(k):

            kimg = (kmap == j)*1
            plt.imsave(f'./kmeans_result_30/outputs/{hdu_date}_{j}.png', kimg, vmin = 0, vmax = 1, cmap = 'gray', origin = 'lower')

        endtime = datetime.now()
        print('coasted ' + str(endtime - starttime))
        
    except Exception as e:
        error_list.append(e)
        print('error:', e)
        continue

data processing


/tmp/ipykernel_2221566/2492021236.py:12: RuntimeWarning: invalid value encountered in sqrt
  cos = abs((1 - (r/rsun)**2)**0.5)


file done


/home/aiia/newDisk/anaconda3/envs/zz_pytorch/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


coasted 0:04:40.028321
data processing


...

coasted 0:04:24.810670
data processing


/tmp/ipykernel_2221566/2492021236.py:12: RuntimeWarning: invalid value encountered in sqrt
  cos = abs((1 - (r/rsun)**2)**0.5)


file done


/home/aiia/newDisk/anaconda3/envs/zz_pytorch/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


coasted 0:04:45.476878
